In [1]:
from pathlib import Path
import sys

PROJECT_ROOT = Path.cwd().resolve().parent
if str(PROJECT_ROOT) not in sys.path:
    sys.path.append(str(PROJECT_ROOT))

In [2]:
from pyspark.sql import functions as F
from pyspark.sql.window import Window

from spark_jobs.config import GOLD_DIR, SILVER_PATH, ensure_data_dirs
from spark_jobs.spark_session_manager import get_spark_session

In [3]:
ensure_data_dirs()
spark = get_spark_session('Gold Layer')

silver_path = Path(SILVER_PATH)
if not silver_path.exists():
    raise FileNotFoundError(f'Silver parquet not found at {silver_path}')

df = spark.read.parquet(str(silver_path))
df.printSchema()

root
 |-- year: integer (nullable = true)
 |-- state: string (nullable = true)
 |-- district: string (nullable = true)
 |-- crop: string (nullable = true)
 |-- season: string (nullable = true)
 |-- data_ingestao: timestamp (nullable = true)
 |-- area: double (nullable = true)
 |-- production: integer (nullable = true)
 |-- yield: double (nullable = true)
 |-- jan: double (nullable = true)
 |-- feb: double (nullable = true)
 |-- mar: double (nullable = true)
 |-- apr: double (nullable = true)
 |-- may: double (nullable = true)
 |-- jun: double (nullable = true)
 |-- jul: double (nullable = true)
 |-- aug: double (nullable = true)
 |-- sep: double (nullable = true)
 |-- oct: double (nullable = true)
 |-- nov: double (nullable = true)
 |-- dec: double (nullable = true)
 |-- ann: double (nullable = true)
 |-- jan_feb: double (nullable = true)
 |-- mar_may: double (nullable = true)
 |-- jun_sep: double (nullable = true)
 |-- oct_dec: double (nullable = true)
 |-- temp_annual: double (nullab

In [4]:
GOLD_DIR_PATH = Path(GOLD_DIR)

def write_parquet(df_out, name: str):
    target = GOLD_DIR_PATH / f'{name}.parquet'
    df_out.write.mode('overwrite').parquet(str(target))
    print(f'Saved {name} -> {target}')

In [5]:
prod_anual = df.groupBy('year', 'crop').agg(
    F.sum('area').alias('area_total_plantada'),
    F.sum('production').alias('producao_total'),
)
prod_anual = prod_anual.withColumn(
    'rendimento_ponderado',
    F.when(
        F.col('area_total_plantada') > 0,
        F.col('producao_total') / F.col('area_total_plantada'),
    ).otherwise(F.lit(0.0)),
)
write_parquet(prod_anual, 'producao_anual_cultura')

Saved producao_anual_cultura -> /home/jovyan/work/data/gold/producao_anual_cultura.parquet


In [6]:
desempenho_regiao = (
    df.groupBy('state_district', 'crop')
    .agg(
        F.sum('area').alias('area_total'),
        F.sum('production').alias('producao_total'),
        F.avg('yield').alias('rendimento_medio_local'),
    )
    .orderBy(F.col('producao_total').desc())
)
write_parquet(desempenho_regiao, 'desempenho_regiao_cultura')

Saved desempenho_regiao_cultura -> /home/jovyan/work/data/gold/desempenho_regiao_cultura.parquet


In [7]:
analise_sazonal = (
    df.groupBy('season', 'crop')
    .agg(
        F.avg('production').alias('producao_media'),
        F.avg('yield').alias('rendimento_medio'),
        F.avg('area').alias('area_media'),
        F.avg('temp_jun_sep').alias('temp_media_jun_sep'),
        F.avg('jun_sep').alias('precipitacao_media_jun_sep'),
    )
    .orderBy(F.col('rendimento_medio').desc())
)
write_parquet(analise_sazonal, 'analise_sazonal_clima')

Saved analise_sazonal_clima -> /home/jovyan/work/data/gold/analise_sazonal_clima.parquet


In [8]:
window_crop_year = Window.partitionBy('crop').orderBy('year')
trend_rendimento = (
    df.groupBy('crop', 'year')
    .agg(F.avg('yield').alias('rendimento_medio'))
    .orderBy('crop', 'year')
)
trend_rendimento = trend_rendimento.withColumn(
    'lag_rendimento', F.lag('rendimento_medio').over(window_crop_year)
)
trend_rendimento = trend_rendimento.withColumn(
    'crescimento_anual_rendimento_percent',
    F.when(
        F.col('lag_rendimento').isNotNull() & (F.col('lag_rendimento') != 0),
        (F.col('rendimento_medio') - F.col('lag_rendimento'))
        / F.col('lag_rendimento')
        * 100,
    ).otherwise(F.lit(0.0)),
).drop('lag_rendimento')
write_parquet(trend_rendimento, 'tendencia_anual_rendimento')

Saved tendencia_anual_rendimento -> /home/jovyan/work/data/gold/tendencia_anual_rendimento.parquet


In [9]:
media_nacional = df.groupBy('crop').agg(
    F.avg('yield').alias('rendimento_medio_nacional')
)
media_regional = df.groupBy('state_district', 'crop').agg(
    F.avg('yield').alias('rendimento_medio_local'),
    F.sum('production').alias('producao_total_local'),
)
benchmark_regional = media_regional.join(media_nacional, on='crop', how='inner')
benchmark_regional = benchmark_regional.withColumn(
    'diferenca_vs_media_percent',
    (F.col('rendimento_medio_local') - F.col('rendimento_medio_nacional'))
    / F.col('rendimento_medio_nacional')
    * 100,
).orderBy(F.col('diferenca_vs_media_percent').desc_nulls_last())
write_parquet(benchmark_regional, 'benchmark_regional_rendimento')

Saved benchmark_regional_rendimento -> /home/jovyan/work/data/gold/benchmark_regional_rendimento.parquet


In [10]:
perfil_climatico = df.groupBy('state_district', 'crop').agg(
    F.avg('yield').alias('rendimento_medio'),
    F.avg('ann').alias('precipitacao_anual_media'),
    F.avg('jun_sep').alias('precipitacao_jun_sep_media'),
    F.avg('temp_jan_feb').alias('temp_media_janfeb'),
    F.avg('temp_mar_may').alias('temp_media_marmay'),
    F.avg('temp_jun_sep').alias('temp_media_junsep'),
    F.avg('temp_oct_dec').alias('temp_media_octdec'),
)
write_parquet(perfil_climatico, 'perfil_climatico_regiao_cultura')

Saved perfil_climatico_regiao_cultura -> /home/jovyan/work/data/gold/perfil_climatico_regiao_cultura.parquet


In [11]:
volatilidade = df.groupBy('state_district', 'crop').agg(
    F.avg('yield').alias('rendimento_medio'),
    F.stddev('yield').alias('rendimento_std_dev'),
    F.count('yield').alias('contagem_registros'),
)
volatilidade = volatilidade.filter(F.col('contagem_registros') >= 3)
volatilidade = volatilidade.withColumn(
    'coef_variacao_rendimento_percent',
    F.when(
        F.col('rendimento_medio') != 0,
        (F.col('rendimento_std_dev') / F.col('rendimento_medio')) * 100,
    ).otherwise(F.lit(0.0)),
).fillna(0)
volatilidade = volatilidade.orderBy(F.col('coef_variacao_rendimento_percent').desc_nulls_last())
write_parquet(volatilidade, 'volatilidade_rendimento_regiao')

Saved volatilidade_rendimento_regiao -> /home/jovyan/work/data/gold/volatilidade_rendimento_regiao.parquet


In [12]:
spark.stop()